In [ ]:
import torch
import torchtext
import torch.nn as nn
import torch.optim as optim

#from torchtext.datasets import Multi30k
#from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time
import pandas as pd

In [ ]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True


In [ ]:

data_s08 = pd.read_csv('datasets/question_answer_pairs_S08.txt',sep='\t')
data_s09 = pd.read_csv('datasets/question_answer_pairs_S09.txt',sep='\t')
data_s10 = pd.read_csv('datasets/question_answer_pairs_S10.txt',sep='\t')

df = data_s08.append(data_s09)
combined = df.append(data_s10)

combined.drop(['DifficultyFromQuestioner','DifficultyFromAnswerer','ArticleFile','ArticleTitle'],axis=1,inplace=True)

combined['Question'] = combined['Question'].str.replace('[.\']','')
combined['Answer'] = combined['Answer'].str.replace('[.\']','')

combined['Answer'] = combined['Answer'].str.lower()
combined['Question'] = combined['Question'].str.lower()

combined.drop_duplicates(keep=False,inplace=True)
combined['Answer'].replace('', np.nan, inplace=True)
combined.dropna(subset=['Answer'], inplace=True)
combined.to_csv( "datasets/qa_dataset.csv", index=False, encoding='utf-8-sig')







In [ ]:
df = pd.read_csv('datasets/qa_dataset.csv')
df

,Question,Answer
0,who suggested lincoln grow a beard?,11-year-old grace bedell
1,who suggested lincoln grow a beard?,grace bedell
2,which county was lincoln born in?,hardin county
3,which county was lincoln born in?,"southeast hardin county, kentucky"
4,who was the general in charge at the battle of...,general mcclellan
...,...,...
2166,what areas do the grevys zebras inhabit?,semi-arid grasslands of ethiopia and northern ...
2167,which species of zebra is known as the common ...,"plains zebra (equus quagga, formerly equus bur..."
2168,which species of zebra is known as the common ...,plains zebra
2169,at what age can a zebra breed?,five or six


In [ ]:

spacy_en = spacy.load('en_core_web_sm')

In [ ]:

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """

    #print([tok.text if not tok.text.isnumeric() else p.number_to_words(tok.text) for tok in spacy_en.tokenizer(text)][::-1])
    return [tok.text for tok in spacy_en.tokenizer(text)][::-1]

In [ ]:
Questions = torchtext.legacy.data.Field(tokenize = tokenize_en,init_token='<sos>',eos_token='<eos>',lower=True)
Answers = torchtext.legacy.data.Field(tokenize =tokenize_en,init_token='<sos>',eos_token='<eos>',lower=True)

In [ ]:
fields = [('Question',Questions),('Answer',Answers)]


In [ ]:
example = [torchtext.legacy.data.Example.fromlist([df.Question[i],df.Answer[i]], fields) for i in range(df.shape[0])] 

In [ ]:
qa_dataset = torchtext.legacy.data.Dataset(example, fields)

In [ ]:
(train_data, valid_data) = qa_dataset.split(split_ratio=[0.70, 0.30], random_state=random.seed(SEED))

In [ ]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")

Number of training examples: 1520
Number of validation examples: 651


In [ ]:
print(vars(train_data.examples[0]))

{'Question': ['?', 'melbourne', 'in', 'majority', 'holds', 'religion', 'what'], 'Answer': ['christian']}


In [ ]:
Questions.build_vocab(train_data, min_freq = 2)
Answers.build_vocab(train_data, min_freq = 2)

In [ ]:
print(f"Unique tokens in questions vocabulary: {len(Questions.vocab)}")
print(f"Unique tokens in answers vocabulary: {len(Answers.vocab)}")

Unique tokens in questions vocabulary: 1517
Unique tokens in answers vocabulary: 1090


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
BATCH_SIZE = 32

train_iterator, valid_iterator = torchtext.legacy.data.BucketIterator.splits(
    (train_data, valid_data), 
    batch_size = BATCH_SIZE,sort_within_batch = True,
    sort_key = lambda x: len(x.Question),
    device = device)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
        
        #input = [batch size]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [seq len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #seq len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.fc_out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden and previous cell states
            #receive output tensor (predictions) and new hidden and cell states
            output, hidden, cell = self.decoder(input, hidden, cell)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1
        
        return outputs

In [ ]:
INPUT_DIM = len(Questions.vocab)
OUTPUT_DIM = len(Answers.vocab)
ENC_EMB_DIM = 300
DEC_EMB_DIM = 300
HID_DIM = 150
N_LAYERS = 2
ENC_DROPOUT = 0.2
DEC_DROPOUT = 0.2
print(INPUT_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

1517


In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08, 0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(1517, 300)
    (rnn): LSTM(300, 150, num_layers=2, dropout=0.2)
    (dropout): Dropout(p=0.2, inplace=False)
  )
  (decoder): Decoder(
    (embedding): Embedding(1090, 300)
    (rnn): LSTM(300, 150, num_layers=2, dropout=0.2)
    (fc_out): Linear(in_features=150, out_features=1090, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
)

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,851,490 trainable parameters


In [ ]:
optimizer = optim.Adam(model.parameters())

In [ ]:
TRG_PAD_IDX = Answers.vocab.stoi[Answers.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.Question
        trg = batch.Answer
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src= batch.Question
            trg= batch.Answer

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
N_EPOCHS = 25
CLIP = 1
train_loss_list = []
valid_loss_list = []
train_ppl_list = []
valid_ppl_list = []
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    train_loss_list.append(train_loss)
    train_ppl_list.append(math.exp(train_loss))

    valid_loss = evaluate(model, valid_iterator, criterion)
    valid_loss_list.append(valid_loss)
    valid_ppl_list.append(math.exp(valid_loss))
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')